<a href="https://colab.research.google.com/github/Ishansingh438/Face-emotion-recognition/blob/main/Ishan_Singh_Face_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Introduction

The Indian education landscape has been undergoing rapid changes for the past 10 years owing to
the advancement of web-based learning services, specifically, eLearning platforms.

Global E-learning is estimated to witness an 8X over the next 5 years to reach USD 2B in 2021. India
is expected to grow with a CAGR of 44% crossing the 10M users mark in 2021. Although the market
is growing on a rapid scale, there are major challenges associated with digital learning when
compared with brick and mortar classrooms. One of many challenges is how to ensure quality
learning for students. Digital platforms might overpower physical classrooms in terms of content
quality but when it comes to understanding whether students are able to grasp the content in a live
class scenario is yet an open-end challenge.

In a physical classroom during a lecturing teacher can see the faces and assess the emotion of the
class and tune their lecture accordingly, whether he is going fast or slow. He can identify students who
need special attention. Digital classrooms are conducted via video telephony software program (exZoom) where it’s not possible for medium scale class (25-50) to see all students and access the
mood. Because of this drawback, students are not focusing on content due to lack of surveillance.
While digital platforms have limitations in terms of physical surveillance but it comes with the power of
data and machines which can work for you. It provides data in the form of video, audio, and texts
which can be analysed using deep learning algorithms. Deep learning backed system not only solves
the surveillance issue, but it also removes the human bias from the system, and all information is no
longer in the teacher’s brain rather translated in numbers that can be analysed and tracked.




# Problem Statements

We will solve the above-mentioned challenge by applying deep learning algorithms to live video data.
The solution to this problem is by recognizing facial emotions.

# Face Emotion Recognition

This is a few shot learning live face emotion detection system. The model should be able to real-time
identify the emotions of students in a live class.


In [ ]:
#Importing required libraries and Dependencies

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
#import utils
import os
%matplotlib inline

tf.keras.preprocessing.image.ImageDataGenerator
tf.keras.layers.Dense
tf.keras.layers.Input
tf.keras.layers.Dropout
tf.keras.layers.Flatten
tf.keras.layers.Conv2D
tf.keras.layers.BatchNormalization 
tf.keras.layers.Activation
tf.keras.layers.MaxPooling2D
tf.keras.Model
tf.keras.Sequential

tf.keras.optimizers.Adam

tf.keras.callbacks.ModelCheckpoint
tf.keras.callbacks.ReduceLROnPlateau
tf.keras.callbacks.EarlyStopping

tf.keras.utils.plot_model

# sklearn.metrics.confusion_matrix
# sklearn.metrics.accuracy_score

import itertools

from IPython.display import SVG, Image
!pip install livelossplot
from livelossplot import PlotLossesKeras

from tensorflow import keras
print("Tensorflow version:", tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.4 MB/s 
Tensorflow version: 2.8.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Train and test data loading

train_dir = ''
test_dir = ''

train_dir = 'train/'
test_dir = 'test/'

# Data Visualization

In [ ]:
# Defining a function to plot some images from different classes

def plot_images(img_dir, top=10):
    all_img_dirs = os.listdir(img_dir)
    img_files = [os.path.join(img_dir, file) for file in all_img_dirs][:5]
  
    plt.figure(figsize=(10, 10))
  
    for idx, img_path in enumerate(img_files):
        plt.subplot(5, 5, idx+1)
    
        img = plt.imread(img_path)
        plt.tight_layout()         
        plt.imshow(img, cmap='Blues_r')

In [ ]:
# angry class
plot_images(train_dir+'angry')

In [ ]:
# disgust class
plot_images(train_dir+'/disgust'

In [ ]:
# fear class
plot_images(train_dir+'/fear')

In [ ]:
# happy class
plot_images(train_dir+'/happy')

In [ ]:
# neutral class
plot_images(train_dir+'/neutral')

In [ ]:
# sad class
plot_images(train_dir+'/sad')

In [ ]:
# surprise class
plot_images(train_dir+'/surprise')

In [ ]:
for expression in os.listdir("train/"):
    print(str(len(os.listdir("train/" + expression))) + " " + expression + " images")

The dataset consists of 2 folders - training and test All the images of dataset are grayscale images of size 48*48. Both these folders consist of 7 folders each.(0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral). The dataset contains approximately 36K images

Dataset is converted into raw images and splitted in multiple folders which are train and validation folders. 80% of our images are contained inside the train folder, and the last 20% are inside the validation folder.

# Data Generators for ResNet

In [ ]:
img_size = 48
batch_size = 32

# selecting colour mode as rgb as resnet is trained on rgb photos and we have grascaled images
datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,brightness_range=[0.8,1.2],rescale=1./255)
train_generator = datagen_train.flow_from_directory(train_dir,
                                                  target_size=(img_size,img_size),
                                                  batch_size=batch_size,
                                                  shuffle=True,
                                                  color_mode='rgb',
                                                  class_mode='categorical')

datagen_validation = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,brightness_range=[0.8,1.2],rescale=1./255)
validation_generator = datagen_train.flow_from_directory(test_dir,
                                                  target_size=(img_size,img_size),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  color_mode='rgb',
                                                  class_mode='categorical')

# Building CNN model with ResNet

In [ ]:
# Using Pretrained model, RESNET50 architecture

tf.keras.applications.resnet50.ResNet50

In [ ]:
# Creating a base model using resnet and loading the pretrained weights
base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(48,48,3),include_top = False, weights = 'imagenet')
base_model.summary()

In [ ]:
# Making all the layers except last 4 layers non trainable 
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [ ]:
# Build model on the top of base model
model = tf.keras.Sequential()

model.add(base_model)
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.BatchNormalization())

#Fully connected 1st layer
model.add(tf.keras.layers.Dense(512,kernel_initializer='he_normal'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.25))


# Fully connected layer 2nd layer
model.add(tf.keras.layers.Dense(256,kernel_initializer='he_normal'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.25))

#output layer
model.add(tf.keras.layers.Dense(7, activation='softmax'))

In [ ]:
# Final model summary
model.summary()

In [ ]:
# Compiling the model

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs=50
steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch

In [ ]:
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps

In [ ]:
# This decreases the learning rate if the model loss does not decrease 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=10, min_lr=0.00001, mode='auto')
# To save model weights 
checkpoint = tf.keras.callbacks.ModelCheckpoint("model_weights_resnet.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=10,
                            verbose=1,
                            restore_best_weights=True)
callbacks = [checkpoint, reduce_lr,early_stopping]

In [ ]:
# Fitting the model 
history = model.fit(
    x=train_generator,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = validation_generator,
    callbacks=callbacks,
    verbose=1)